### Comparison of optimizers for LASSO parameter range experiments

sklearn has 2 ways to fit logistic regression models with a LASSO penalty: [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) (using the liblinear optimizer; i.e. coordinate descent), and [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) (which uses stochastic gradient descent).

Here, we want to compare mutation prediction results between the two optimizers, across all the cancer genes in our driver gene set.

In [1]:
import os
import itertools as it

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
import seaborn as sns

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.data_utilities as du
import pancancer_evaluation.utilities.analysis_utilities as au

%load_ext autoreload
%autoreload 2

In [2]:
ll_base_results_dir = os.path.join(
    cfg.repo_root, '02_cancer_type_classification', 'results', 'lasso_range_lr_all_features'
)

# this doesn't have a sex covariate but it's probably close enough
sgd_base_results_dir = os.path.join(
    cfg.repo_root, '02_cancer_type_classification', 'results', 'lasso_range_valid'
)

training_dataset = 'all_other_cancers'
ll_results_dir = os.path.join(ll_base_results_dir, training_dataset)
sgd_results_dir = os.path.join(sgd_base_results_dir, training_dataset)

metric = 'aupr'
test_gene = 'EGFR' # TODO: remove after testing

output_plots = False
output_plots_dir = None

### Get coefficient information for each lasso penalty

In [3]:
# these are generated from results files pretty slowly so it helps to cache them
ll_coefs_df_file = './ll_coefficients_df.tsv'

if os.path.exists(ll_coefs_df_file):
    print('exists')
    ll_nz_coefs_df = pd.read_csv(ll_coefs_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    ll_nz_coefs_df = []
    # get coefficient info for training dataset specified above
    for coef_info in au.generate_nonzero_coefficients_lasso_range(ll_results_dir,
                                                                  gene=test_gene):
        (gene,
         cancer_type,
         seed,
         lasso_param,
         coefs_list) = coef_info
        for fold_no, coefs in enumerate(coefs_list):
            ll_nz_coefs_df.append(
                [gene, cancer_type, lasso_param, seed, fold_no, len(coefs)]
            )
    ll_nz_coefs_df = pd.DataFrame(
        ll_nz_coefs_df,
        columns=['gene', 'cancer_type', 'lasso_param', 'seed', 'fold', 'nz_coefs']
    )
    ll_nz_coefs_df.lasso_param = ll_nz_coefs_df.lasso_param.astype(float)
    ll_nz_coefs_df.to_csv(ll_coefs_df_file, sep='\t')
                                                                  
ll_nz_coefs_df.head()

exists


,gene,cancer_type,lasso_param,seed,fold,nz_coefs
0,EGFR,ESCA,0.01,1,0,38
1,EGFR,ESCA,0.01,1,1,36
2,EGFR,ESCA,0.01,1,2,35
3,EGFR,ESCA,0.01,1,3,32
4,EGFR,ESCA,750.00,1,0,12923


In [4]:
# these are generated from results files pretty slowly so it helps to cache them
sgd_coefs_df_file = './sgd_coefficients_df.tsv'

if os.path.exists(sgd_coefs_df_file):
    print('exists')
    sgd_nz_coefs_df = pd.read_csv(sgd_coefs_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    sgd_nz_coefs_df = []
    # get coefficient info for training dataset specified above
    for coef_info in au.generate_nonzero_coefficients_lasso_range(sgd_results_dir,
                                                                  gene=test_gene):
        (gene,
         cancer_type,
         seed,
         lasso_param,
         coefs_list) = coef_info
        for fold_no, coefs in enumerate(coefs_list):
            sgd_nz_coefs_df.append(
                [gene, cancer_type, lasso_param, seed, fold_no, len(coefs)]
            )
    sgd_nz_coefs_df = pd.DataFrame(
        sgd_nz_coefs_df,
        columns=['gene', 'cancer_type', 'lasso_param', 'seed', 'fold', 'nz_coefs']
    )
    sgd_nz_coefs_df.lasso_param = sgd_nz_coefs_df.lasso_param.astype(float)
    sgd_nz_coefs_df.to_csv(sgd_coefs_df_file, sep='\t')
                                                                  
sgd_nz_coefs_df.head()

exists


,gene,cancer_type,lasso_param,seed,fold,nz_coefs
0,EGFR,LGG,0.00025,42,0,7428
1,EGFR,LGG,0.00025,42,1,7507
2,EGFR,LGG,0.00025,42,2,7560
3,EGFR,LGG,0.00025,42,3,7517
4,EGFR,HNSC,0.00025,42,0,7391


### Get performance information for each lasso penalty

In [5]:
# load performance information
ll_perf_df_file = './ll_perf_df.tsv'

if os.path.exists(ll_perf_df_file):
    print('exists')
    ll_perf_df = pd.read_csv(ll_perf_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    ll_perf_df = au.load_prediction_results_lasso_range(ll_results_dir,
                                                        'liblinear',
                                                        gene=test_gene)
    ll_perf_df.rename(columns={'experiment': 'optimizer'}, inplace=True)
    ll_perf_df.lasso_param = ll_perf_df.lasso_param.astype(float)
    ll_perf_df.to_csv(ll_perf_df_file, sep='\t')

ll_perf_df.head()

exists


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param
0,0.87778,0.72319,EGFR,LGG,signal,42,train,0,liblinear,0.005
1,0.83249,0.68721,EGFR,LGG,signal,42,test,0,liblinear,0.005
2,0.87023,0.69618,EGFR,LGG,signal,42,cv,0,liblinear,0.005
3,0.87304,0.70521,EGFR,LGG,signal,42,train,1,liblinear,0.005
4,0.80657,0.70045,EGFR,LGG,signal,42,test,1,liblinear,0.005


In [6]:
# add nonzero coefficient count
ll_plot_df = (
    ll_perf_df[(ll_perf_df.signal == 'signal')]
      .merge(ll_nz_coefs_df, left_on=['holdout_cancer_type', 'lasso_param', 'seed', 'fold'],
             right_on=['cancer_type', 'lasso_param', 'seed', 'fold'])
      .drop(columns=['cancer_type', 'gene_y'])
      .rename(columns={'gene_x': 'gene'})
      .sort_values(by=['holdout_cancer_type', 'lasso_param'])
      .reset_index(drop=True)
)
ll_plot_df.lasso_param = ll_plot_df.lasso_param.astype(float)

print(ll_plot_df.shape)
ll_plot_df.head()

(3072, 11)


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param,nz_coefs
0,0.5,0.128400,EGFR,BLCA,signal,42,train,0,liblinear,0.001,0
1,0.5,0.081633,EGFR,BLCA,signal,42,test,0,liblinear,0.001,0
2,0.5,0.132390,EGFR,BLCA,signal,42,cv,0,liblinear,0.001,0
3,0.5,0.130180,EGFR,BLCA,signal,42,train,1,liblinear,0.001,0
4,0.5,0.081633,EGFR,BLCA,signal,42,test,1,liblinear,0.001,0


In [7]:
# load performance information
sgd_perf_df_file = './sgd_perf_df.tsv'

if os.path.exists(sgd_perf_df_file):
    print('exists')
    sgd_perf_df = pd.read_csv(sgd_perf_df_file, sep='\t', index_col=0)
else:
    print('not exists')
    sgd_perf_df = au.load_prediction_results_lasso_range(sgd_results_dir,
                                                        'sgd',
                                                        gene=test_gene)
    sgd_perf_df.rename(columns={'experiment': 'optimizer'}, inplace=True)
    sgd_perf_df.lasso_param = sgd_perf_df.lasso_param.astype(float)
    sgd_perf_df.to_csv(sgd_perf_df_file, sep='\t')

sgd_perf_df.head()

exists


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param
0,0.82408,0.38905,EGFR,STAD,signal,1,train,0,sgd,0.1
1,0.76809,0.39725,EGFR,STAD,signal,1,test,0,sgd,0.1
2,0.80928,0.44742,EGFR,STAD,signal,1,cv,0,sgd,0.1
3,0.80149,0.47194,EGFR,STAD,signal,1,train,1,sgd,0.1
4,0.71667,0.21007,EGFR,STAD,signal,1,test,1,sgd,0.1


In [8]:
# add nonzero coefficient count
sgd_plot_df = (
    sgd_perf_df[(sgd_perf_df.signal == 'signal')]
      .merge(sgd_nz_coefs_df, left_on=['holdout_cancer_type', 'lasso_param', 'seed', 'fold'],
             right_on=['cancer_type', 'lasso_param', 'seed', 'fold'])
      .drop(columns=['cancer_type', 'gene_y'])
      .rename(columns={'gene_x': 'gene'})
      .sort_values(by=['holdout_cancer_type', 'lasso_param'])
      .reset_index(drop=True)
)
sgd_plot_df.lasso_param = sgd_plot_df.lasso_param.astype(float)

print(sgd_plot_df.shape)
sgd_plot_df.head()

(1920, 11)


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param,nz_coefs
0,0.99992,0.99946,EGFR,BLCA,signal,42,train,0,sgd,0.0001,7727
1,0.78750,0.49195,EGFR,BLCA,signal,42,test,0,sgd,0.0001,7727
2,0.81917,0.58199,EGFR,BLCA,signal,42,cv,0,sgd,0.0001,7727
3,0.99998,0.99988,EGFR,BLCA,signal,42,train,1,sgd,0.0001,7617
4,0.71667,0.44997,EGFR,BLCA,signal,42,test,1,sgd,0.0001,7617


In [9]:
all_perf_df = pd.concat((ll_plot_df, sgd_plot_df)).reset_index(drop=True)

print(all_perf_df.shape)
print(all_perf_df.optimizer.unique())
all_perf_df.head()

(4992, 11)
['liblinear' 'sgd']


,auroc,aupr,gene,holdout_cancer_type,signal,seed,data_type,fold,optimizer,lasso_param,nz_coefs
0,0.5,0.128400,EGFR,BLCA,signal,42,train,0,liblinear,0.001,0
1,0.5,0.081633,EGFR,BLCA,signal,42,test,0,liblinear,0.001,0
2,0.5,0.132390,EGFR,BLCA,signal,42,cv,0,liblinear,0.001,0
3,0.5,0.130180,EGFR,BLCA,signal,42,train,1,liblinear,0.001,0
4,0.5,0.081633,EGFR,BLCA,signal,42,test,1,liblinear,0.001,0


### Select best lasso parameter for each optimizer

We'll do this for both CV (validation) datasets and test (holdout cancer type) datasets.

In [10]:
# get mean AUPR values across folds/seeds
ll_mean_aupr_df = (
    all_perf_df[(all_perf_df.data_type == 'cv') &
                (all_perf_df.optimizer == 'liblinear')]
      .drop(columns=['data_type', 'optimizer'])
      .groupby(['gene', 'holdout_cancer_type', 'lasso_param'])
      .agg(np.mean)
      .reset_index()
      .drop(columns=['seed', 'fold', 'auroc', 'nz_coefs'])
)

# get best LASSO parameter by mean AUPR, across all the ones we tried for this optimizer
ll_max_lasso_ix = (ll_mean_aupr_df
      .groupby(['gene', 'holdout_cancer_type'])
      .aupr.idxmax()
)
ll_max_lasso_param_df = ll_mean_aupr_df.loc[ll_max_lasso_ix, :]

print(ll_max_lasso_param_df.shape)
ll_max_lasso_param_df.head(8)

(8, 4)


,gene,holdout_cancer_type,lasso_param,aupr
2,EGFR,BLCA,0.010,0.752182
18,EGFR,ESCA,0.010,0.766919
34,EGFR,GBM,0.010,0.716318
50,EGFR,HNSC,0.010,0.751248
66,EGFR,LGG,0.010,0.743325
81,EGFR,LUAD,0.005,0.785305
98,EGFR,LUSC,0.010,0.767281
114,EGFR,STAD,0.010,0.751846


In [11]:
# get mean AUPR values across folds/seeds
sgd_mean_aupr_df = (
    all_perf_df[(all_perf_df.data_type == 'cv') &
                (all_perf_df.optimizer == 'sgd')]
      .drop(columns=['data_type', 'optimizer'])
      .groupby(['gene', 'holdout_cancer_type', 'lasso_param'])
      .agg(np.mean)
      .reset_index()
      .drop(columns=['seed', 'fold', 'auroc', 'nz_coefs'])
)

# get best LASSO parameter by mean AUPR, across all the ones we tried for this optimizer
sgd_max_lasso_ix = (sgd_mean_aupr_df
      .groupby(['gene', 'holdout_cancer_type'])
      .aupr.idxmax()
)
sgd_max_lasso_param_df = sgd_mean_aupr_df.loc[sgd_max_lasso_ix, :]

print(sgd_max_lasso_param_df.shape)
sgd_max_lasso_param_df.head(8)

(8, 4)


,gene,holdout_cancer_type,lasso_param,aupr
0,EGFR,BLCA,0.0001,0.568958
10,EGFR,ESCA,0.0001,0.532741
20,EGFR,GBM,0.0001,0.503974
30,EGFR,HNSC,0.0001,0.564353
45,EGFR,LGG,0.0050,0.520886
56,EGFR,LUAD,0.0100,0.588788
60,EGFR,LUSC,0.0001,0.550624
76,EGFR,STAD,0.0100,0.561978
